In [4]:
# Relevant module imports and installs
import pandas as pd
!pip install pulp brotli
import pulp as plp
import sys 
import os
from collections import defaultdict


[notice] A new release of pip is available: 23.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
# Get the absolute path to the directory containing the Python file
module_path = os.path.abspath(os.path.join('..', '..'))

# Add the directory to sys.path
if module_path not in sys.path:
    sys.path.append(module_path)

# Now you can import the module
from projections import generate_projections, generate_stat_projections, append_stat_projections

point_projections = generate_projections()
stat_projections = generate_stat_projections()

projections_data = append_stat_projections(point_projections, stat_projections, 2)

# For all predicted points, if position is Forward, multiply the value by 1.7, round to 2dp
projections_data.loc[projections_data['Position'] == 'Forward', 'Predicted_Points'] *= 1.7
projections_data['Predicted_Points'] = projections_data['Predicted_Points'].round(2)

# Print any players' index here
# print index of Haaland
print(projections_data[projections_data['Name'] == 'Haaland'].index[0])

165


,ID,Name,Team,Position,Cost,Predicted_Points,xMins,Opponent,Score,Assist,Goal_Involvement,Clean_Sheet
0,2,G.Jesus,Arsenal,Forward,6.9,0.44,9,Aston Villa (A),0.188,0.102,0.271,0.205
1,3,Gabriel,Arsenal,Defender,6.0,4.22,90,Aston Villa (A),0.123,0.075,0.188,0.341
2,4,Havertz,Arsenal,Forward,8.1,8.76,90,Aston Villa (A),0.256,0.128,0.352,0.341
3,6,J.Timber,Arsenal,Defender,5.5,0.01,5,Aston Villa (A),0.000,0.000,0.000,0.000
4,9,Martinelli,Arsenal,Midfielder,7.0,4.64,75,Aston Villa (A),0.238,0.167,0.365,0.341
...,...,...,...,...,...,...,...,...,...,...,...,...
305,619,Cajuste,Ipswich,Midfielder,4.5,0.00,0,Man City (A),0.000,0.000,0.000,0.000
306,620,Sosa,Nott'm Forest,Midfielder,5.5,0.00,0,Southampton (A),0.000,0.000,0.000,0.000
307,621,João Félix,Chelsea,Midfielder,6.5,0.00,0,Wolves (A),0.000,0.000,0.000,0.000
308,622,Berge,Fulham,Midfielder,5.0,2.86,90,Leicester (H),0.000,0.000,0.000,0.000


# 2024/25 GW2 Challenge: All Out Attack - Goals are worth double points. Your squad may only contain 1 Goalkeeper and 4 Forwards.

In [6]:
# Get the number of players and their list of ids
player_ids = projections_data['ID'].tolist()
player_count = len(player_ids)

# Set up the problem
model = plp.LpProblem("fpl-gw1-challenge", plp.LpMaximize)

# Define the decision variables
lineup = [
    plp.LpVariable(f"lineup_{i}", lowBound=0, upBound=1, cat="Integer")
    for i in player_ids
]

# Define captain variables
captain = [
    plp.LpVariable(f"captain_{i}", lowBound=0, upBound=1, cat="Integer")
    for i in player_ids
]

# Set the objective function (the number of points scored by the team, with captain's points doubled)
model += plp.lpSum([lineup[i] * projections_data.loc[i, 'Predicted_Points'] for i in range(player_count)]) + \
         plp.lpSum([captain[i] * projections_data.loc[i, 'Predicted_Points'] for i in range(player_count)])

# Constraints

# Total number of players = 5 (1 goalkeeper + 4 forwards)
model += plp.lpSum(lineup) == 5

# List players by index to be EXCLUDED from the lineup
# Haaland = 165
model += lineup[165] == 0

# Exactly one captain
model += plp.lpSum(captain) == 1

# Captain must be in the lineup
for i in range(player_count):
    model += captain[i] <= lineup[i]

# Exactly 1 Goalkeeper
model += plp.lpSum([lineup[i] for i in range(player_count) if projections_data.loc[i, 'Position'] == 'Goalkeeper']) == 1

# Exactly 4 Forwards
model += plp.lpSum([lineup[i] for i in range(player_count) if projections_data.loc[i, 'Position'] == 'Forward']) == 4

# Budget constraint: Total cost must be less than or equal to 35m
model += plp.lpSum([lineup[i] * projections_data.loc[i, 'Cost'] for i in range(player_count)]) <= 35

# Solve the problem
model.solve()

# Function to print players by position
def print_players_by_position(players_dict):
    total_points = 0
    total_cost = 0
    for position in ['Goalkeeper', 'Forward']:
        if position in players_dict:
            print(f"\n{position}:")
            for player in players_dict[position]:
                captain_str = " (C)" if player['Captain'] else ""
                points = player['Predicted_Points'] * (2 if player['Captain'] else 1)
                print(f"  {player['Name']}{captain_str} - {player['Team']} - Cost: {player['Cost']}m - Predicted Points: {points}")
                total_points += points
                total_cost += player['Cost']
    print(f"\nTotal Predicted Points: {round(total_points, 2)}")
    print(f"Total Cost: {round(total_cost, 2)}m")

# Print the results
print("Status:", plp.LpStatus[model.status])
selected_players = defaultdict(list)
for i in range(player_count):
    if lineup[i].value() == 1:
        player = projections_data.loc[i]
        selected_players[player['Position']].append({
            'Name': player['Name'],
            'Team': player['Team'],
            'Cost': player['Cost'],
            'Predicted_Points': player['Predicted_Points'],
            'Captain': captain[i].value() == 1
        })

print_solution = False
if print_solution:
    print("\nOptimal Lineup:")
    print_players_by_position(selected_players)

Status: Optimal

Optimal Lineup:

Goalkeeper:
  Ederson M. - Man City - Cost: 5.5m - Predicted Points: 4.61

Forward:
  Havertz - Arsenal - Cost: 8.1m - Predicted Points: 8.76
  João Pedro - Brighton - Cost: 5.5m - Predicted Points: 8.11
  Muniz - Fulham - Cost: 6.0m - Predicted Points: 9.15
  Isak (C) - Newcastle - Cost: 8.5m - Predicted Points: 20.34

Total Predicted Points: 50.97
Total Cost: 33.6m
